<h1>Chapter 8 - Semantic Search and Retrieval-Augmented Generation</h1>
<i>Exploring a vital part of LLMs, search.</i>

<a href="https://www.amazon.com/Hands-Large-Language-Models-Understanding/dp/1098150961"><img src="https://img.shields.io/badge/Buy%20the%20Book!-grey?logo=amazon"></a>
<a href="https://www.oreilly.com/library/view/hands-on-large-language/9781098150952/"><img src="https://img.shields.io/badge/O'Reilly-white.svg?logo=data:image/svg%2bxml;base64,PHN2ZyB3aWR0aD0iMzQiIGhlaWdodD0iMjciIHZpZXdCb3g9IjAgMCAzNCAyNyIgZmlsbD0ibm9uZSIgeG1sbnM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDAvc3ZnIj4KPGNpcmNsZSBjeD0iMTMiIGN5PSIxNCIgcj0iMTEiIHN0cm9rZT0iI0Q0MDEwMSIgc3Ryb2tlLXdpZHRoPSI0Ii8+CjxjaXJjbGUgY3g9IjMwLjUiIGN5PSIzLjUiIHI9IjMuNSIgZmlsbD0iI0Q0MDEwMSIvPgo8L3N2Zz4K"></a>
<a href="https://github.com/HandsOnLLM/Hands-On-Large-Language-Models"><img src="https://img.shields.io/badge/GitHub%20Repository-black?logo=github"></a>
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/HandsOnLLM/Hands-On-Large-Language-Models/blob/main/chapter08/Chapter%208%20-%20Semantic%20Search.ipynb)



### [OPTIONAL] - Installing Packages on <img src="https://colab.google/static/images/icons/colab.png" width=100>

If you are viewing this notebook on Google Colab (or any other cloud vendor), you need to **uncomment and run** the following codeblock to install the dependencies for this chapter:

---

💡 **NOTE**: We will want to use a GPU to run the examples in this notebook. In Google Colab, go to
**Runtime > Change runtime type > Hardware accelerator > GPU > GPU type > T4**.

---


In [ ]:
# %%capture
!pip install langchain==0.2.5 faiss-cpu==1.8.0 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1 pandas python-dotenv
!pip install llama-cpp-python==0.2.78  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

## IMPORTANT: Make sure to restart the session after installing the packages above.

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


## Setup

In [ ]:
import cohere
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variable
api_key = os.environ.get('COHERE_API_KEY')

# Verify the API key was loaded
if api_key is None:
    print("⚠️ WARNING: COHERE_API_KEY not found in environment variables!")
    print("Please paste your API key below or add it to the .env file")
    # Uncomment the line below and paste your API key if .env doesn't work
    # api_key = 'your-api-key-here'
else:
    print("✅ API key loaded successfully!")
    print(f"API key starts with: {api_key[:10]}...")

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

ApiError: status_code: None, body: The client must be instantiated be either passing in token or setting CO_API_KEY

In [ ]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = [t.strip(' \n') for t in texts]

In [ ]:
import numpy as np

# Get the embeddings
response = co.embed(
  texts=texts,
  input_type="search_document", # this means we use the embeddings to store in a vector database.
).embeddings

embeds = np.array(response)
print(embeds.shape)

In [ ]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim) # Uses L2 Euclidean distance metric
index.add(np.float32(embeds))

In [ ]:
import pandas as pd

def search(query, number_of_results=5):

  # 1. Get the query's embedding
  query_embed = co.embed(texts=[query],
                input_type="search_query",).embeddings[0]

  # 2. Retrieve the nearest neighbors
  distances , similar_item_ids = index.search(np.float32([query_embed]), number_of_results)

  # 3. Format the results
  texts_np = np.array(texts) # Convert texts list to numpy for easier indexing
  results = pd.DataFrame(data={'texts': texts_np[similar_item_ids[0]],
                              'distance': distances[0]})

  # 4. Print and return the results
  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [ ]:
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction import _stop_words
import string
from tqdm import tqdm

def bm25_tokenizer(text):
    tokenized_doc = []
    for token in text.lower().split():
        token = token.strip(string.punctuation)

        if len(token) > 0 and token not in _stop_words.ENGLISH_STOP_WORDS:
            tokenized_doc.append(token)
    return tokenized_doc

# Build BM25 index
tokenized_corpus = []
for passage in tqdm(texts):
    tokenized_corpus.append(bm25_tokenizer(passage))

bm25 = BM25Okapi(tokenized_corpus)

def keyword_search(query, top_k=3, num_candidates=15):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

In [ ]:
def keyword_and_reranking_search(query, top_k=3, num_candidates=10):
    print("Input question:", query)

    ##### BM25 search (lexical search) #####
    bm25_scores = bm25.get_scores(bm25_tokenizer(query))
    top_n = np.argpartition(bm25_scores, -num_candidates)[-num_candidates:]
    bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)

    print(f"Top-3 lexical search (BM25) hits")
    for hit in bm25_hits[0:top_k]:
        print("\t{:.3f}\t{}".format(hit['score'], texts[hit['corpus_id']].replace("\n", " ")))

    #Add re-ranking
    docs = [texts[hit['corpus_id']] for hit in bm25_hits]

    print(f"\nTop-3 hits by rank-API ({len(bm25_hits)} BM25 hits re-ranked)")
    results = co.rerank(query=query, documents=docs, top_n=top_k, return_documents=True)
    for hit in results.results:
        print("\t{:.3f}\t{}".format(hit.relevance_score, hit.document.text.replace("\n", " ")))

---

## Easy Tasks

Easy tasks to practice chapter 8 concepts

### Task 1: Testing Different Queries with Semantic Search

**Objective:** Experiment with different types of queries to understand how semantic search works.

**Instructions:**
1. Use the `search()` function with at least 3 different queries
2. Try queries about:
   - Awards/nominations
   - Filming locations
   - Actors
3. Observe the distance scores 

In [2]:
# Task 1: Your code here
# Test at least 3 different queries

# Example query 1: About awards
query1 = "awards and nominations"
results1 = search(query1, number_of_results=3)
print("\nQuery 1 Results:")
print(results1)

# TODO: Add your own queries below
# query2 = 
# query3 = 

NameError: name 'search' is not defined

**Question:** Why do some queries return better results than others?

## Task 2: Compare Dense Retrieval vs BM25

**Objective:** Compare the results from semantic search (dense retrieval) and keyword search (BM25).

**Instructions:**
1. Choose a query and run it with both methods:
   - `search(query)` for dense retrieval
   - `keyword_search(query)` for BM25
2. Compare the top 3 results from each method
3. Are they the same? Different? Why?

**Example queries to try:**
- "Christopher Nolan director"
- "box office revenue"
- "visual effects"

In [ ]:
# Task 2: Your code here
# Compare dense retrieval vs BM25

query = "box office revenue"

print("=" * 50)
print("DENSE RETRIEVAL (Semantic Search):")
print("=" * 50)
dense_results = search(query, number_of_results=3)
print(dense_results)

print("\n" + "=" * 50)
print("BM25 (Keyword Search):")
print("=" * 50)
keyword_search(query, top_k=3)

# TODO: Try with different queries and analyze the differences

## Task 3: Understanding Distance Metrics

**Objective:** Analyze how distance scores indicate relevance in semantic search.

**Instructions:**
1. Run a search query and examine the distance values
2. What does a smaller distance mean?
3. What does a larger distance mean?
4. Find the query that gives you the lowest average distance (most relevant results)

**Hint:** Lower distance = more similar/relevant in L2 distance metric

In [ ]:
# Task 3: Your code here
# Analyze distance metrics

# Test multiple queries and compare average distances
queries_to_test = [
    "science fiction film",
    "quantum physics",  # Not relevant to Interstellar content
    "Christopher Nolan"
]

for q in queries_to_test:
    results = search(q, number_of_results=3)
    avg_distance = results['distance'].mean()
    print(f"\nQuery: '{q}'")
    print(f"Average distance: {avg_distance:.4f}")
    print(f"Distance range: {results['distance'].min():.4f} to {results['distance'].max():.4f}")

# TODO: What patterns do you notice about relevant vs irrelevant queries?

## Task 4: Observe the Impact of Reranking

**Objective:** See how reranking improves search results.

**Instructions:**
1. Use `keyword_and_reranking_search()` with different queries
2. Compare the BM25 scores before reranking with relevance scores after reranking
3. Does the order change? By how much?
4. Which document gets the highest relevance score?

**Example queries:**
- "scientific accuracy physics"
- "movie release date"
- "actors cast members"

In [ ]:
# Task 4: Your code here
# Observe reranking impact

query = "scientific accuracy physics"
print(f"Testing reranking with query: '{query}'\n")
keyword_and_reranking_search(query, top_k=3, num_candidates=10)

print("\n" + "="*60)
print("\nAnalysis Questions:")
print("1. Did the order change after reranking?")
print("2. Which method (BM25 or reranked) seems more accurate?")
print("3. What's the relevance score of the top result?")

# TODO: Test with other queries

## Task 5: Test RAG with Different Questions

**Objective:** Understand how RAG combines retrieval and generation.

**Instructions:**
1. Use the `rag.invoke()` function with at least 3 questions
2. Try questions that:
   - Have clear answers in the text
   - Require combining information from multiple sentences
   - Are NOT answerable from the text
3. Observe how the model responds in each case

**Sample questions:**
- "When did Interstellar premiere?"
- "What awards did the movie win?"
- "What is the capital of France?" (not in the text!)

In [ ]:
# Task 5: Your code here
# Test RAG with different questions

questions = [
    "When did Interstellar premiere?",
    "What awards did the movie win?",
    "Who wrote the screenplay?"
]

for question in questions:
    print("\n" + "="*60)
    print(f"Question: {question}")
    print("="*60)
    result = rag.invoke(question)
    print(f"\nAnswer: {result['result']}")

# TODO: Add a question that CANNOT be answered from the text
# How does the model respond?

## Task 6: Explore Embedding Dimensions

**Objective:** Understand the structure of embeddings.

**Instructions:**
1. Print the shape of the embeddings
2. How many dimensions does each embedding have?
3. Print the embedding of a single sentence
4. What do these numbers represent?

**Key concepts:**
- Each sentence is converted to a vector of numbers
- The dimensionality determines the "resolution" of semantic meaning

In [ ]:
# Task 6: Your code here
# Explore embedding dimensions

print("Embedding shape:", embeds.shape)
print(f"Number of documents: {embeds.shape[0]}")
print(f"Embedding dimensions: {embeds.shape[1]}")

# Look at a single embedding
print("\nFirst sentence:", texts[0])
print(f"\nIts embedding (first 10 dimensions): {embeds[0][:10]}")
print(f"Embedding min value: {embeds[0].min():.4f}")
print(f"Embedding max value: {embeds[0].max():.4f}")

# TODO: What do you think these numbers represent?
# TODO: Why might more dimensions be useful?

## Task 7: Experiment with Different Chunking Strategies

**Objective:** See how text chunking affects search quality.

**Instructions:**
1. Create a new chunking strategy (e.g., split by '!' or by number of words)
2. Create embeddings for the new chunks
3. Build a new search index
4. Test if search results are better or worse

**Consider:**
- What happens if chunks are too small?
- What happens if chunks are too large?
- What's the optimal chunk size?

In [ ]:
# Task 7: Your code here
# Experiment with different chunking strategies

# Option 1: Split by newlines instead of periods
texts_by_paragraph = [t.strip() for t in text.split('\n\n') if t.strip()]
print(f"Original chunking: {len(texts)} chunks")
print(f"Paragraph chunking: {len(texts_by_paragraph)} chunks")

# Option 2: Create fixed-size chunks (by number of words)
def chunk_by_words(text, words_per_chunk=20):
    words = text.split()
    chunks = []
    for i in range(0, len(words), words_per_chunk):
        chunk = ' '.join(words[i:i + words_per_chunk])
        chunks.append(chunk)
    return chunks

texts_by_words = chunk_by_words(text, words_per_chunk=30)
print(f"Word-based chunking (30 words): {len(texts_by_words)} chunks")

# TODO: Create embeddings for one of these alternative chunking strategies
# TODO: Compare search results with the original chunking

## Task 8: Reflection Questions

**Answer these questions based on your experiments:**

1. **Dense Retrieval vs BM25:**
   - When would you use dense retrieval (semantic search)?
   - When would you use BM25 (keyword search)?
   - What are the advantages of each?

2. **Reranking:**
   - How much does reranking improve results?
   - Is it worth the extra computational cost?
   - When would reranking be most valuable?

3. **RAG (Retrieval-Augmented Generation):**
   - How does RAG help reduce hallucinations?
   - What happens when the answer isn't in the retrieved documents?
   - What are the limitations of RAG?

4. **Embedding Models:**
   - Why do we need the same embedding model for documents and queries?
   - What would happen if we used different models?
   - How do `input_type="search_document"` and `input_type="search_query"` differ?

5. **Practical Applications:**
   - What real-world applications could benefit from semantic search?
   - How would you build a RAG system for a company's internal documents?
   - What challenges might you face in production?

### Your Answers:

**Write your reflections here...**

1. Dense Retrieval vs BM25:
   - 

2. Reranking:
   - 

3. RAG:
   - 

4. Embedding Models:
   - 

5. Practical Applications:
   - 

---

## 🎉 Congratulations!

You've completed the Easy Tasks for Chapter 8! 

**Next Steps:**
- Review your answers to the reflection questions
- Try the Medium Tasks notebook for more advanced challenges
- Experiment with your own text corpus
- Build a simple RAG application for a specific domain

**Key Takeaways:**
- Semantic search uses embeddings to find similar content
- BM25 uses keyword matching for lexical search
- Reranking improves result quality
- RAG combines retrieval and generation to ground LLM responses
- Chunking strategy significantly impacts search quality